## Funciones de costos

Durante esta lección, aprenderemos cómo evaluar una *función de costos* :

- Primero, aprenderemos sobre [las primitivas de Qiskit Runtime.](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html)
- Defina una *función de costo* $C(\vec\theta)$. Esta es una función específica del problema que define el objetivo del problema para que el optimizador minimice (o maximice)
- Definición de una estrategia de medición con las primitivas de Qiskit Runtime para optimizar la velocidad frente a la precisión

&nbsp;

![Flujo de trabajo de la función de costos](images/cost_function_workflow.png)

## Primitivos

Todos los sistemas físicos, ya sean clásicos o cuánticos, pueden existir en diferentes estados. Por ejemplo, un automóvil en una carretera puede tener una determinada masa, posición, velocidad o aceleración que caractericen su estado. De manera similar, los sistemas cuánticos también pueden tener diferentes configuraciones o estados, pero se diferencian de los sistemas clásicos en la forma en que manejamos las mediciones y la evolución de los estados. Esto conduce a propiedades únicas, como *la superposición* y *el entrelazamiento* , que son exclusivas de la mecánica cuántica. Así como podemos describir el estado de un automóvil usando propiedades físicas como la velocidad o la aceleración, también podemos describir el estado de un sistema cuántico usando *observables* , que son objetos matemáticos.

En mecánica cuántica, los estados están representados por vectores de columna complejos normalizados, o *kets* ($|\psi\rangle$), y los observables son operadores lineales hermitianos ($\hat{H}=\hat{H}^{\dagger}$ ) que actúan sobre los kets. Un vector propio ($|\lambda\rangle$) de un observable se conoce como *estado propio* . Medir un observable para uno de sus estados propios ($|\lambda\rangle$) nos dará el valor propio correspondiente ($\lambda$) como lectura.

Si se pregunta cómo medir un sistema cuántico y qué se puede medir, Qiskit ofrece dos [primitivos](gloss:primitives) que pueden ayudar:

- `Sampler` : Dado un estado cuántico $|\psi\rangle$, esta primitiva obtiene la probabilidad de cada posible estado base computacional.
- `Estimator` : Dado un observable cuántico $\hat{H}$ y un estado $|\psi\rangle$, esta primitiva calcula el valor esperado de $\hat{H}$.


### La primitiva Sampler

La primitiva `Sampler` calcula la probabilidad de obtener cada estado posible $|k\rangle$ a partir de la base computacional, dado un circuito cuántico que prepara el estado $|\psi\rangle$. calcula

$$
p_k = |\langle k | \psi \rangle|^2 \quad \forall k \in \mathbb{Z}_2^n \equiv \{0,1,\cdots,2^n-1\},
$$

Donde $n$ es el número de qubits y $k$ la representación entera de cualquier posible cadena binaria de salida ${0,1}^n$ (es decir, números enteros con base $2$).

`Sampler` [de Qiskit Runtime](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Sampler.html) ejecuta el circuito varias veces en un dispositivo cuántico, realiza mediciones en cada ejecución y reconstruye la distribución de probabilidad a partir de las cadenas de bits recuperadas. Cuantas más ejecuciones (o *disparos* ) realice, más precisos serán los resultados, pero esto requiere más tiempo y recursos cuánticos.

Sin embargo, dado que el número de salidas posibles crece exponencialmente con el número de qubits $n$ (es decir, $2^n$), el número de disparos también deberá crecer exponencialmente para capturar una distribución de probabilidad *densa* . Por lo tanto, `Sampler` sólo es eficiente para distribuciones de probabilidad *escasas* ; donde el estado objetivo $|\psi\rangle$ debe poder expresarse como una combinación lineal de los estados básicos computacionales, con el número de términos creciendo como máximo polinomialmente con el número de qubits:

$$
|\psi\rangle = \sum^{\text{Poly}(n)}_k w_k |k\rangle.
$$

El `Sampler` también se puede configurar para recuperar probabilidades de una subsección del circuito, que representa un subconjunto del total de estados posibles.

### El estimador primitivo

La primitiva `Estimator` calcula el valor esperado de un $\hat{H}$ observable para un estado cuántico $|\psi\rangle$; donde las probabilidades observables se pueden expresar como $p_\lambda = |\langle\lambda|\psi\rangle|^2$, siendo $|\lambda\rangle$ los estados propios del observable $\hat{H}$. El valor esperado se define entonces como el promedio de todos los resultados posibles $\lambda$ (es decir, los valores propios de lo observable) de una medición del estado $|\psi\rangle$, ponderados por las probabilidades correspondientes:

$$
\langle\hat{H}\rangle_\psi := \sum_\lambda p_\lambda \lambda = \langle \psi | \hat{H} | \psi \rangle
$$

Sin embargo, no siempre es posible calcular el valor esperado de un observable, ya que a menudo no conocemos su base propia. `Estimator` [de Qiskit Runtime](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Estimator.html) utiliza un proceso algebraico complejo para estimar el valor esperado en un dispositivo cuántico real al descomponer lo observable en una combinación de otros observables cuya base propia conocemos.

En términos más simples, `Estimator` descompone cualquier observable que no sabe cómo medir en observables más simples y mensurables llamados [operadores de Pauli](gloss:pauli) .

Cualquier operador se puede expresar como una combinación de operadores de Pauli $4^n$.

$$
\hat{P}_k := 
\sigma_{k_{n-1}}\otimes \cdots \otimes \sigma_{k_0} \quad 
\forall k \in \mathbb{Z}_4^n \equiv \{0,1,\cdots,4^n-1\}, \\
$$

tal que

$$
\hat{H} = \sum^{4^n-1}_{k=0} w_k \hat{P}_k
$$

donde $n$ es el número de qubits, $k \equiv k_{n-1} \cdots k_0$ para $k_l \in \mathbb{Z}_4 \equiv {0, 1, 2, 3}$ (es decir, números enteros base $4$), y $(\sigma_0, \sigma_1, \sigma_2, \sigma_3) := (I, X, Y, Z)$.

Después de realizar esta descomposición, `Estimator` deriva un nuevo circuito $V_k|\psi\rangle$ para cada $\hat{P}_k$ observable (es decir, del circuito original), para *diagonalizar* efectivamente el observable de Pauli en la base computacional y medirlo. . Podemos medir fácilmente los observables de Pauli porque conocemos $V_k$ de antemano, lo que generalmente no ocurre con otros observables.

Para cada `{latex} \hat{P}_{k}` , el `Estimator` ejecuta el circuito correspondiente en un dispositivo cuántico varias veces, mide el estado de salida en la base computacional y calcula la probabilidad $p_{kj}$ de obtener cada posible salida $j$. Luego busca el valor propio $\lambda_{kj}$ de $P_k$ correspondiente a cada salida $j$, lo multiplica por $w_k$ y suma todos los resultados para obtener el valor esperado del observable $\hat{H }$ para el estado dado $|\psi\rangle$.

$$
\langle\hat{H}\rangle_\psi = 
\sum_{k=0}^{4^n-1} w_k \sum_{j=0}^{2^n-1}p_{kj} \lambda_{kj},
$$

Dado que calcular el valor esperado de $4^n$ Paulis no es práctico (es decir, crece exponencialmente), `Estimator` solo puede ser eficiente cuando una gran cantidad de $w_k$ es cero (es decir, descomposición *escasa* de Pauli en lugar de *densa* ). Formalmente decimos que, para que este cálculo se pueda *resolver de manera eficiente* , el número de términos distintos de cero tiene que crecer como máximo polinomialmente con el número de qubits $n$: $\hat{H} = \sum^{\text{Poly }(n)}_k w_k \hat{P}_k.$

El lector puede notar la suposición implícita de que [el muestreo](gloss:sampling) probabilístico también debe ser eficiente como se explica en `Sampler` , lo que significa

$$
\langle\hat{H}\rangle_\psi = 
\sum_{k}^{\text{Poly}(n)} w_k \sum_{j}^{\text{Poly}(n)}p_{kj} \lambda_{kj}.
$$

### Ejemplo guiado para calcular los valores esperados

Supongamos que el estado de un solo qubit $|+\rangle := H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$, y observable

$$
\begin{aligned}
\hat{H}
& = \begin{pmatrix} 
-1 & 2 \\
2 & -1 \\
\end{pmatrix}\\[1mm]
& = 2X - Z

\end{aligned}
$$

con el siguiente valor teórico esperado $\langle\hat{H}\rangle_+ = \langle+|\hat{H}|+\rangle = 2.$

Como no sabemos cómo medir este observable, no podemos calcular su valor esperado directamente y necesitamos reexpresarlo como $\langle\hat{H}\rangle_+ = 2\langle X \rangle_+ - \langle Z \rangle_+ $. Se puede demostrar que se evalúa el mismo resultado al observar que $\langle+|X|+\rangle = 1$ y $\langle+|Z|+\rangle = 0$.

Veamos cómo calcular $\langle X \rangle_+$ y $\langle Z \rangle_+$ directamente. Dado que $X$ y $Z$ no conmutan (es decir, no comparten la misma base propia), no se pueden medir simultáneamente, por lo tanto necesitamos los circuitos auxiliares:

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# The following code will work for any other initial single-qubit state and observable
original_circuit = QuantumCircuit(1)
original_circuit.h(0)

H = SparsePauliOp(["X", "Z"], [2, -1])

aux_circuits = []
for pauli in H.paulis:
    aux_circ = original_circuit.copy()
    aux_circ.barrier()
    if str(pauli) == "X":
        aux_circ.h(0)
    elif str(pauli) == "Y":
        aux_circ.sdg(0)
        aux_circ.h(0)
    else:
        aux_circ.i(0)
    aux_circ.measure_all()
    aux_circuits.append(aux_circ)


print("Original circuit:")
display(original_circuit.draw("mpl"))
for (circuit, pauli) in zip(aux_circuits, H.paulis):
    print(f"Auxiliary circuit for {str(pauli)}")
    display(circuit.draw("mpl"))

Ahora podemos realizar el cálculo manualmente usando `Sampler` y verificar los resultados en `Estimator` :

In [ ]:
from qiskit.primitives import Sampler, Estimator
from qiskit.circuit.library import IGate, ZGate
import numpy as np


## SAMPLER
sampler = Sampler()
job = sampler.run(aux_circuits)
probability_dists = job.result().quasi_dists

expvals = []
for dist, pauli in zip(probability_dists, H.paulis):
    val = 0
    if str(pauli) == "I":
        Lambda = IGate().to_matrix().real
    else:
        Lambda = ZGate().to_matrix().real
    val += Lambda[0][0] * dist.get(0, 0)
    val += Lambda[1][1] * dist.get(1, 0)
    expvals.append(val)


print("Sampler results:")
for (pauli, expval) in zip(H.paulis, expvals):
    print(f"  >> Expected value of {str(pauli)}: {expval:.5f}")

total_expval = np.sum(H.coeffs * expvals).real
print(f"  >> Total expected value: {total_expval:.5f}")


## ESTIMATOR
observables = [
    *H.paulis,
    H,
]  # Note: run for individual Paulis as well as full observable H

estimator = Estimator()
job = estimator.run([original_circuit] * len(observables), observables)
estimator_expvals = job.result().values

print("Estimator results:")
for (obs, expval) in zip(observables, estimator_expvals):
    if obs is not H:
        print(f"  >> Expected value of {str(obs)}: {expval:.5f}")
    else:
        print(f"  >> Total expected value: {expval:.5f}")

### Rigor matemático (opcional)

Expresando $|\psi\rangle$ con respecto a la base de estados propios de $\hat{H}$, $|\psi\rangle = \sum_\lambda a_\lambda |\lambda\rangle$, se sigue:

$$
\begin{aligned}
\langle \psi | \hat{H} | \psi \rangle
& = \bigg(\sum_{\lambda'}a^*_{\lambda'} \langle \lambda'|\bigg) \hat{H} 
  \bigg(\sum_{\lambda} a_\lambda | \lambda\rangle\bigg)\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} 
  \langle \lambda'|\hat{H}| \lambda\rangle\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} \lambda 
\langle \lambda'| \lambda\rangle\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} \lambda 
\cdot \delta_{\lambda, \lambda'}\\[1mm]

& = \sum_\lambda |a_\lambda|^2 \lambda\\[1mm]

& = \sum_\lambda p_\lambda \lambda\\[1mm]

\end{aligned}
$$

Como no conocemos los valores propios o estados propios del objetivo observable $\hat{H}$, primero debemos considerar su diagonalización. Dado que $\hat{H}$ es [hermitiano](gloss:hermitian) , existe una transformación unitaria $V$ tal que $\hat{H}=V^\dagger \Lambda V,$ donde $\Lambda$ es la matriz de valores propios diagonal, entonces $\langle j | \Lambda | k \rangle = 0$ si $j\neq k$, y $\langle j | \Lambda | j \rangle = \lambda_j$.

Esto implica que el valor esperado se puede reescribir como:

$$
\begin{aligned}
\langle\psi|\hat{H}|\psi\rangle
& = \langle\psi|V^\dagger \Lambda V|\psi\rangle\\[1mm]

& = \langle\psi|V^\dagger \bigg(\sum_{j=0}^{2^n-1} |j\rangle 
\langle j|\bigg) \Lambda \bigg(\sum_{k=0}^{2^n-1} |k\rangle \langle k|\bigg) V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1} \sum_{k=0}^{2^n-1}\langle\psi|V^\dagger |j\rangle 
\langle j| \Lambda  |k\rangle \langle k| V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1}\langle\psi|V^\dagger |j\rangle 
\langle j| \Lambda  |j\rangle \langle j| V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1}|\langle j| V|\psi\rangle|^2 \lambda_j\\[1mm]

\end{aligned}
$$

Dado que si un sistema está en el estado $|\phi\rangle = V |\psi\rangle$ la probabilidad de medir $| j\rangle$ es $p_j = |\langle j|\phi \rangle|^2$, el valor esperado anterior se puede expresar como:

$$
\langle\psi|\hat{H}|\psi\rangle = 
\sum_{j=0}^{2^n-1} p_j \lambda_j.
$$

Es muy importante tener en cuenta que las probabilidades se toman del estado $V |\psi\rangle$ en lugar de $|\psi\rangle$. Por eso la matriz $V$ es absolutamente necesaria.

Quizás se pregunte cómo obtener la matriz $V$ y los valores propios $\Lambda$. Si ya tuviera los valores propios, entonces no habría necesidad de usar una computadora cuántica ya que el objetivo de los algoritmos variacionales es encontrar estos valores propios de $\hat{H}$.

Afortunadamente, hay una manera de evitarlo: cualquier matriz $2^n \times 2^n$ se puede escribir como una combinación lineal de productos tensoriales $4^n$ de matrices e identidades de Pauli $n$, todas las cuales son hermitianas y unitario con $V$ y $\Lambda$ conocidos. Esto es lo que hace internamente `Estimator` de Runtime al descomponer cualquier objeto [Operador](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Operator.html) en un [SparsePauliOp](https://qiskit.org/documentation/stubs/qiskit.quantum_info.SparsePauliOp.html) .

Estos son los operadores que se pueden utilizar:

$$
\begin{array}{c|c|c|c}
  \text{Operator} & \sigma & V & \Lambda \\[1mm]
  \hline
  I & \sigma_0 = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} & V_0 = I & \Lambda_0 = I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \\[4mm]

  X & \sigma_1 = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} & V_1 = H =\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix} & \Lambda_1 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \\[4mm]

  Y & \sigma_2 = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} & V_2 = HS^\dagger  =\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}\cdot  \begin{pmatrix} 1 & 0 \\ 0 & -i \end{pmatrix} = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & -i \\ 1 & i \end{pmatrix}\quad & \Lambda_2 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \\[4mm]

  Z & \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} & V_3 = I & \Lambda_3 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
\end{array}
$$

Así que reescribamos $\hat{H}$ con respecto a Paulis y las identidades:

$$
\hat{H} = 
\sum_{k_{n-1}=0}^3...
\sum_{k_0=0}^3 w_{k_{n-1}...k_0} 
\sigma_{k_{n-1}}\otimes ... \otimes \sigma_{k_0} = \sum_{k=0}^{4^n-1} w_k \hat{P}_k,
$$

donde $k = \sum_{l=0}^{n-1} 4^l k_l \equiv k_{n-1}...k_0$ para $k_{n-1},...,k_0\in {0,1,2,3}$ (es decir, base $4$), y `{latex} \hat{P}_{k} := \sigma_{k_{n-1}}\otimes ... \otimes \sigma_{k_0}` :

$$
\begin{aligned}
\langle\psi|\hat{H}|\psi\rangle
& = \sum_{k=0}^{4^n-1} w_k 
\sum_{j=0}^{2^n-1}|\langle j| V_k|\psi\rangle|^2 \langle j| \Lambda_k |j\rangle \\[1mm]

& = \sum_{k=0}^{4^n-1} w_k \sum_{j=0}^{2^n-1}p_{kj} \lambda_{kj}, \\[1mm]


\end{aligned}
$$

donde $V_k := V_{k_{n-1}}\otimes ... \otimes V_{k_0}$ y $\Lambda_k := \Lambda_{k_{n-1}}\otimes ... \otimes \ Lambda_{k_0}$, tal que: $\hat{P_k}=V_k^\dagger \Lambda_k V_k.$

## Funciones de costos

En general, las funciones de costos se utilizan para describir el objetivo de un problema y qué tan bien se está desempeñando un estado de prueba con respecto a ese objetivo. Esta definición se puede aplicar a varios ejemplos de química, aprendizaje automático, finanzas, optimización, etc.

Consideremos un ejemplo sencillo de cómo encontrar el estado fundamental de un sistema. Nuestro objetivo es minimizar el valor esperado del observable que representa la energía (hamiltoniano $\hat{\mathcal{H}}$):

$$
\min_{\vec\theta} \langle\psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle
$$

Podemos usar la [primitiva *Estimador*](https://github.com/qiskit-community/prototype-zne/blob/main/docs/tutorials/0-estimator.ipynb) para evaluar el valor esperado y pasar este valor a un optimizador para minimizarlo. Si la optimización tiene éxito, devolverá un conjunto de valores de parámetros óptimos $\vec\theta^ *$, a partir del cual podremos construir el estado de solución propuesto $|\psi(\vec\theta^* )\rangle$ y calcule el valor esperado observado como $C(\vec\theta^*)$.

Observe cómo sólo podremos minimizar la función de costos para el conjunto limitado de estados que estamos considerando. Esto nos lleva a dos posibilidades separadas:

- **Nuestro ansatz no define el estado de la solución en el espacio de búsqueda** : si este es el caso, nuestro optimizador nunca encontrará la solución y necesitamos experimentar con otros ansatz que puedan representar nuestro espacio de búsqueda con mayor precisión.
- **Nuestro optimizador no puede encontrar esta solución válida** : la optimización se puede definir globalmente y localmente. Exploraremos lo que esto significa en la sección posterior.

En definitiva, realizaremos un ciclo de optimización clásico pero basándonos en la evaluación de la función de costes de una computadora cuántica. Desde esta perspectiva, uno podría pensar en la optimización como un esfuerzo puramente clásico en el que llamamos a algún [*oráculo cuántico de caja negra*](gloss:oracle) cada vez que el optimizador necesita evaluar la función de costos.

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

# Add your token below
service = QiskitRuntimeService(channel="ibm_quantum")

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("XX", 1), ("YY", -3)])

    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values


theta_list = (2 * np.pi * np.random.rand(1, 8)).tolist()
cost_function_vqe(theta_list)

### Ejemplo de mapeo a sistemas no físicos

El problema de corte máximo (Max-Cut) es un problema de optimización combinatoria que implica dividir los vértices de un gráfico en dos conjuntos disjuntos de modo que se maximice el número de aristas entre los dos conjuntos. Más formalmente, dado un gráfico no dirigido $G=(V,E)$, donde $V$ es el conjunto de vértices y $E$ es el conjunto de aristas, el problema Max-Cut pide dividir los vértices en dos subconjuntos disjuntos , $S$ y $T$, de modo que se maximiza el número de aristas con un punto final en $S$ y el otro en $T$.

Podemos aplicar Max-Cut para resolver varios problemas que incluyen: agrupamiento, diseño de redes, transiciones de fase, etc. Comenzaremos creando un gráfico de problemas:

In [ ]:
import networkx as nx

n = 4
G = nx.Graph()
G.add_nodes_from(range(n))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
G.add_weighted_edges_from(edge_list)

colors = ["red" for i in range(n)]


def draw_graph(G, colors):
    """Draws the graph with the chose colors"""
    layout = nx.shell_layout(G)
    nx.draw_networkx(G, node_color=colors, pos=layout)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)


draw_graph(G, colors)

Este problema se puede expresar como un problema de optimización binaria. Para cada nodo $0 \leq i &lt; n$, donde $n$ es el número de nodos del gráfico (en este caso $n=4$), consideraremos la variable binaria $x_i$. Esta variable tendrá el valor $1$ si el nodo $i$ es uno de los grupos que etiquetaremos $1$ y $0$ si está en el otro grupo, que etiquetaremos como $0$. También denotaremos como $w_{ij}$ (elemento $(i,j)$ de la matriz de adyacencia $w$) el peso de la arista que va del nodo $i$ al nodo $j$. Debido a que la gráfica no está dirigida, $w_{ij}=w_{ji}$. Entonces podemos formular nuestro problema maximizando la siguiente función de costos:

$$
\begin{aligned}
C(\vec{x})
& =\sum_{i,j=0}^n w_{ij} x_i(1-x_j)\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i,j=0}^n w_{ij} x_ix_j\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i=0}^n \sum_{j=0}^i 2w_{ij} x_ix_j
\end{aligned}
$$

Para resolver este problema con una computadora cuántica, vamos a expresar la función de costos como el valor esperado de un observable. Sin embargo, los observables que Qiskit admite de forma nativa consisten en operadores de Pauli, que tienen valores propios $1$ y $-1$ en lugar de $0$ y $1$. Por eso vamos a realizar el siguiente cambio de variable:

Donde $\vec{x}=(x_0,x_1,\cdots,x_{n-1})$. Podemos utilizar la matriz de adyacencia $w$ para acceder cómodamente a los pesos de todas las aristas. Esto se utilizará para obtener nuestra función de costos:

$$
z_i = 1-2x_i \rightarrow x_i = \frac{1-z_i}{2}
$$

Esto implica que:

$$
\begin{array}{lcl} x_i=0 & \rightarrow & z_i=1 \\ x_i=1 & \rightarrow & z_i=-1.\end{array}
$$

Entonces la nueva función de costos que queremos maximizar es:

$$
\begin{aligned}
C(\vec{z})
& = \sum_{i,j=0}^n w_{ij} \bigg(\frac{1-z_i}{2}\bigg)\bigg(1-\frac{1-z_j}{2}\bigg)\\[1mm]

& = \sum_{i,j=0}^n \frac{w_{ij}}{4} - \sum_{i,j=0}^n \frac{w_{ij}}{4} z_iz_j\\[1mm]

& = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j
\end{aligned}
$$

Además, la tendencia natural de una computadora cuántica es encontrar mínimos (generalmente la energía más baja) en lugar de máximos, por lo que en lugar de maximizar $C(\vec{z})$ vamos a minimizar:

$$
-C(\vec{z}) =  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Ahora que tenemos una función de costo para minimizar cuyas variables pueden tener los valores $-1$ y $1$, podemos hacer la siguiente analogía con la $Z$ de Pauli:

$$
z_i \equiv Z_i = \overbrace{I}^{n-1}\otimes ... \otimes \overbrace{Z}^{i} \otimes ... \otimes \overbrace{I}^{0}
$$

En otras palabras, la variable $z_i$ será equivalente a una puerta $Z$ que actúa sobre el qubit $i$. Además:

$$
Z_i|x_{n-1}\cdots x_0\rangle = z_i|x_{n-1}\cdots x_0\rangle \rightarrow \langle x_{n-1}\cdots x_0 |Z_i|x_{n-1}\cdots x_0\rangle = z_i
$$

Entonces el observable que vamos a considerar es:

$$
\hat{H} = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} Z_iZ_j
$$

a lo que tendremos que añadirle después el término independiente:

$$
\texttt{offset} = - \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Esta transformación se puede realizar con [`QuadraticProgram.to_ising()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.QuadraticProgram.to_ising.html) .

In [ ]:
from qiskit_optimization.applications import Maxcut
from qiskit.algorithms.optimizers import COBYLA

w = nx.to_numpy_array(G)

max_cut = Maxcut(w)
quadratic_program = max_cut.to_quadratic_program()
observable, offset = quadratic_program.to_ising()


def cost_function_max_cut_vqe(theta):

    ansatz = TwoLocal(
        observable.num_qubits, "rx", reps=1
    )

    backend = service.backend("ibmq_qasm_simulator")

    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values


optimizer = COBYLA()

initial_theta = np.ones(8)

optimizer_result = optimizer.minimize(fun=cost_function_max_cut_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

Revisaremos este ejemplo en Aplicaciones para explorar cómo aprovechar un optimizador para recorrer el espacio de búsqueda. En términos generales, esto incluye:

- Aprovechar un optimizador para encontrar parámetros óptimos
- Vincular parámetros óptimos al ansatz para encontrar los valores propios
- Traducir los valores propios a nuestra definición de problema

## Estrategia de medición: velocidad versus precisión

Como se mencionó, estamos utilizando una computadora cuántica ruidosa como un *oráculo de caja negra* , donde el ruido puede hacer que los valores recuperados no sean deterministas, lo que lleva a fluctuaciones aleatorias que, a su vez, perjudicarán (o incluso impedirán por completo) la convergencia de ciertos optimizadores a una solución propuesta. Este es un problema general que debemos abordar a medida que avanzamos gradualmente hacia la ventaja cuántica:

![Ventaja](images/cost_function_path_to_quantum_advantage.png)

Podemos utilizar las opciones de supresión y mitigación de errores de Qiskit Runtime Primitive para abordar el ruido y maximizar la utilidad de las computadoras cuánticas actuales.

### Supresión de errores

[La supresión de errores](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-suppression.html) se refiere a técnicas utilizadas para optimizar y transformar un circuito durante la compilación para minimizar los errores. Esta es una técnica básica de manejo de errores que generalmente resulta en una [sobrecarga](gloss:overhead) de preprocesamiento clásica para el tiempo de ejecución general. Los gastos generales incluyen la transpilación de circuitos para ejecutarlos en hardware cuántico mediante:

- Expresando el circuito usando las puertas nativas disponibles en un sistema cuántico
- Mapeo de qubits virtuales a qubits físicos
- Agregar SWAP según los requisitos de conectividad
- Optimización de las puertas 1Q y 2Q
- Agregar desacoplamiento dinámico a los qubits inactivos para evitar los efectos de la decoherencia.

Las primitivas permiten el uso de [técnicas de supresión de errores](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-suppression.html) configurando la opción `optimization_level` y seleccionando opciones de transpilación avanzadas. En un curso posterior, profundizaremos en diferentes métodos de construcción de circuitos para mejorar los resultados, pero para la mayoría de los casos, recomendaríamos configurar `optimization_level=3` .

### Mitigación de errores

[La mitigación de errores](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html) se refiere a técnicas que permiten a los usuarios reducir los errores de circuito modelando el ruido del dispositivo en el momento de la ejecución. Normalmente, esto da como resultado una sobrecarga de preprocesamiento cuántico relacionada con el entrenamiento del modelo y una sobrecarga de posprocesamiento clásica para mitigar errores en los resultados sin procesar mediante el uso del modelo generado.

La opción `resilience_level` de la primitiva Qiskit Runtime especifica la cantidad de resiliencia que se debe desarrollar contra errores. Los niveles más altos generan resultados más precisos a expensas de tiempos de procesamiento más prolongados debido a la sobrecarga del muestreo cuántico. Los niveles de resiliencia se pueden utilizar para configurar el equilibrio entre costo y precisión al aplicar la mitigación de errores a su consulta primitiva.

Al implementar cualquier técnica de mitigación de errores, esperamos que el [sesgo](gloss:bias) en nuestros resultados se reduzca con respecto al sesgo no mitigado anterior. En algunos casos, el sesgo puede incluso desaparecer. Sin embargo, esto tiene un costo. A medida que reducimos el sesgo en nuestras cantidades estimadas, la variabilidad estadística aumentará (es decir, la varianza), lo que podemos explicar aumentando aún más el número de disparos por circuito en nuestro proceso de muestreo. Esto introducirá una sobrecarga superior a la necesaria para reducir el sesgo, por lo que no se realiza de forma predeterminada. Podemos optar fácilmente por este comportamiento ajustando el número de disparos por circuito en options.executions.shots, como se muestra en el siguiente ejemplo.

![Compensación de variación de sesgo](images/cost_function_bias_variance_trade_off.png)

Para este curso, exploraremos estos modelos de mitigación de errores en un alto nivel para ilustrar la mitigación de errores que las primitivas de Qiskit Runtime pueden realizar sin requerir detalles completos de implementación.

### Extinción de errores de lectura girada (T-REx)

La extinción de errores de lectura giratoria (T-REx) utiliza una técnica conocida como giro de Pauli para reducir el ruido introducido durante el proceso de medición cuántica. Esta técnica no asume ninguna forma específica de ruido, lo que la hace muy general y efectiva.

Flujo de trabajo general:

1. Adquirir datos para el estado cero con cambios de bits aleatorios (Pauli X antes de la medición)
2. Adquiera datos para el estado deseado (ruidoso) con cambios de bits aleatorios (Pauli X antes de la medición)
3. Calcule la función especial para cada conjunto de datos y divida.

&nbsp;

![TIRANO SAURIO REX](images/cost_function_trex_data_collection.png)

Podemos configurar esto con `options.resilience_level = 1` , como se demuestra en el siguiente ejemplo.

### Extrapolación de ruido cero

La extrapolación de ruido cero (ZNE) funciona amplificando primero el ruido en el circuito que está preparando el estado cuántico deseado, obteniendo mediciones para varios niveles diferentes de ruido y utilizando esas mediciones para inferir el resultado sin ruido.

Flujo de trabajo general:

1. Amplifica el ruido del circuito para varios factores de ruido.
2. Ejecuta cada circuito amplificado de ruido.
3. Extrapolar nuevamente al límite de ruido cero

&nbsp;

![ZNE](images/cost_function_zne_stages.png)

Podemos configurar esto con `options.resilience_level = 2` . Podemos optimizar esto aún más explorando una variedad de `noise_factors` , `noise_amplifiers` y `extrapolators` , pero esto está fuera del alcance de este curso. Le recomendamos que experimente con estas [opciones como se describe aquí](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html#advanced-resilience-options) .

### Cancelación de error probabilístico

Muestras de cancelación de errores probabilísticos (PEC) para un conjunto de circuitos que, en promedio, imitan un canal de inversión de ruido para cancelar el ruido en el cálculo deseado. Este proceso es un poco parecido al funcionamiento de los auriculares con cancelación de ruido y produce excelentes resultados. Sin embargo, no es tan general como otros métodos y la sobrecarga de muestreo es exponencial.

Flujo de trabajo general:

![PEC](images/cost_function_pec_layers.png)

Paso 1: Pauli girando

![PEC girando](images/cost_function_pec_pauli_twirling.png)

Paso 2: repite la capa y aprende el ruido.

![Aprendizaje PEC](images/cost_function_pec_learn_layer.png)

Paso 3: derivar una fidelidad (error para cada canal de ruido)

![Fidelidad PEC](images/cost_function_pec_curve_fitting.png)

Cada método viene con su propia sobrecarga asociada: una compensación entre la cantidad de cálculos cuánticos necesarios (tiempo) y la precisión de nuestros resultados:

$$
\begin{array}{c|c|c|c}
  \text{Methods} & R=1 \text{, T-REx} & R=2 \text{, ZNE} & R=3 \text{, PEC} \\[1mm]
  \hline
  \text{Assumptions} & \text{None} & \text{Ability to scale noise} & \text{Full knowledge of noise} \\[1mm]
  \text{Qubit overhead} & 1 & 1 & 1 \\[1mm]
  \text{Sampling overhead} & 2 & N_{\text{noise-factors}} & \mathcal{O}(e^{\lambda N_{layers}}) \\[1mm]
  \text{Bias} & 0 & \mathcal{O}(\lambda^{N_{\text{noise-factors}}}) & 0 \\[1mm]
\end{array}
$$

### Uso de las opciones de mitigación y supresión de Qiskit Runtime

A continuación se explica cómo calcular un valor esperado mientras se utiliza la mitigación y supresión de errores en Qiskit Runtime. Este proceso ocurre varias veces a lo largo de un ciclo de optimización, pero hemos mantenido el ejemplo simple para demostrar cómo configurar la mitigación y supresión de errores.

In [ ]:
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

theta = Parameter("theta")

qc = QuantumCircuit(2)
qc.x(1)
qc.h(0)
qc.cp(theta, 0, 1)
qc.h(0)
observables = SparsePauliOp.from_list([("ZZ", 1)])

qc.draw("mpl")

In [ ]:
## Setup phases
import numpy as np

phases = np.linspace(0, 2 * np.pi, 50)

# phases need to be expressed as a list of lists in order to work
individual_phases = [[phase] for phase in phases]

In [ ]:
## Create noise model
from qiskit.providers.fake_provider import FakeManila
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import Options


# Make a noise model
fake_backend = FakeManila()
noise_model = NoiseModel.from_backend(fake_backend)

# Set options to include the noise model
options = Options()
options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.configuration().basis_gates,
    "coupling_map": fake_backend.configuration().coupling_map,
    "seed_simulator": 42,
}

In [ ]:
from qiskit_ibm_runtime import Session, QiskitRuntimeService, Estimator

# Add your token below
service = QiskitRuntimeService(channel="ibm_quantum")

noisy_exp_values = []
exp_values_with_em_es = []

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    # generate noisy results
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    noisy_exp_values = result.values

    options.optimization_level = 2
    options.resilience_level = 1

    # leverage mitigation and suppression
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    exp_values_with_em_es = result.values

    session.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(phases, noisy_exp_values, "o", label="Noisy")
plt.plot(phases, exp_values_with_em_es, "o", label="Mitigation + Suppression")
plt.plot(phases, 2 * np.sin(phases / 2) ** 2 - 1, label="Theory")
plt.xlabel("Phase")
plt.ylabel("Expectation")
plt.legend()
plt.show()

Con esta lección, aprendiste cómo crear una función de costos:

- Crear una función de costo
- Cómo aprovechar [las primitivas de Qiskit Runtime](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html) para mitigar y suprimir el ruido
- Cómo definir una estrategia de medición para optimizar la velocidad frente a la precisión

Aquí está nuestra carga de trabajo variacional de alto nivel:

![Circuito de función de costo](images/cost_function_circuit.png)

Nuestra función de costos se ejecuta durante cada iteración del ciclo de optimización. La próxima lección explorará cómo el optimizador clásico utiliza nuestra evaluación de la función de costos para seleccionar nuevos parámetros.